In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2Config
from transformers import AutoConfig, AutoModelWithLMHead
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import random
from torch.utils.data import random_split
import pandas as pd

In [2]:
tokenizer = AutoTokenizer.from_pretrained(
    "HooshvareLab/gpt2-fa",
    bos_token='<s>',
    eos_token='</s>',
    pad_token='<pad>'
)
tokenizer.add_special_tokens({
    "bos_token": '<s>',
    "eos_token": '</s>',
    "pad_token": '<pad>'
})

configuration = AutoConfig.from_pretrained(
    "HooshvareLab/gpt2-fa",
    bos_token_id=tokenizer("<s>")["input_ids"][0],
    eos_token_id=tokenizer("</s>")["input_ids"][0],
    pad_token_id=tokenizer("<pad>")["input_ids"][0]
)
model = AutoModelForCausalLM.from_pretrained("HooshvareLab/gpt2-fa", config=configuration)
model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/875k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.75M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/14.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/104 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/485M [00:00<?, ?B/s]

Embedding(42001, 768)

## Dataset handling <br>
We separate the different lines, and as the number of "mesraa'"s are odd, we omit the final row. After that, we display what we intend to achieve in the next cell, but the final logic is in the dataset class. Note that according to the question, we have to give a "beyt" as input, and the next "beyt" as output. So we have a second splitting, which is perfomed in the --init-- method

In [3]:
df=pd.read_csv("ferdousi.txt")
df1=df[df.index%2==0]
df2=df[df.index%2==1]
df1=df1.drop(index=0)
df1.reset_index(drop=True,inplace=True)
df2.reset_index(drop=True,inplace=True)

In [4]:
df2=df2.drop(index=len(df2)-1)

In [5]:
s=df1['ferdousi.txt']+"<sep>"+df2['ferdousi.txt']
s.head()


0    کزین برتر اندیشه برنگذرد<sep>به نام خداوند جان...
1    خداوند روزی ده رهنمای<sep>خداوند نام و خداوند جای
2    فروزنده ماه و ناهید و مهر<sep>خداوند کیوان و گ...
3    نگارندهٔ بر شده پیکرست<sep>ز نام و نشان و گمان...
4    نبینی مرنجان دو بیننده را<sep>به بینندگان آفری...
Name: ferdousi.txt, dtype: object

In [6]:
class MyDataset(Dataset):
    def __init__(self, input_df , output_df , tokenizer, max_length=1024):

        self.tokenizer = tokenizer
        #self.tokenizer.pad_token = tokenizer.eos_token
        self.input_ids = []
        self.attn_masks = []
        self.output_ids= []

        input_df.reset_index(drop=True,inplace=True)
        output_df.reset_index(drop=True,inplace=True)
        n=min(len(input_df),len(output_df))
        if len(input_df)>len(output_df):
            input_df=input_df.drop(index=len(input_df)-1)
        elif len(output_df)>len(input_df):
            output_df=output_df.drop(index=len(output_df)-1)

        series_in = input_df.iloc[:, 0]
        series_out = output_df.iloc[:, 0]

        beyt_series=series_in+"<sep>"+series_out
        beyt_df= beyt_series.to_frame(name="a")
        beyt_in=(beyt_df[beyt_df.index%2==0])
        beyt_out=(beyt_df[beyt_df.index%2==1])
        beyt_in.reset_index(drop=True,inplace=True)
        beyt_in_s=beyt_in.iloc[:,0]
        beyt_out.reset_index(drop=True,inplace=True)
        beyt_out_s=beyt_out.iloc[:,0]

        n=len(beyt_in_s)
        for i in range (n):
            in_dict = tokenizer(self.tokenizer.bos_token + beyt_in_s[i] + self.tokenizer.eos_token,
                                       truncation=True,
                                       max_length=max_length,
                                       padding="max_length")
            out_dict = tokenizer(self.tokenizer.bos_token + beyt_out_s[i] + self.tokenizer.eos_token,
                                       truncation=True,
                                       max_length=max_length,
                                       padding="max_length")
            self.input_ids.append(torch.tensor(in_dict.input_ids))
            self.attn_masks.append(torch.tensor(in_dict.attention_mask))
            self.output_ids.append(torch.tensor(out_dict.input_ids))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx], self.output_ids[idx]

In [7]:
dataset = MyDataset(df2,df1,tokenizer, max_length=80)
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [8]:
train_loader = DataLoader(train_dataset, batch_size=50)
test_loader = DataLoader(test_dataset, batch_size=8)
optimizer=torch.optim.Adam(model.parameters())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Training loop

In [9]:
model = model.to(device)
epochs=3
model.train()
for epoch in range(epochs):
    Sum_loss= 0.0
    print("Epoch: %i"%(epoch+1))
    for i,data in enumerate(train_loader,0):
        (input_ids, attn_masks, output_ids)=data
        input_ids=input_ids.to(device)
        output_ids=output_ids.to(device)
        attn_masks=attn_masks.to(device)
        outputs = model(input_ids, labels=output_ids, attention_mask=attn_masks)

        loss = outputs.loss
        Sum_loss += loss.item()/len(train_loader)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f'loss_TRAIN ={Sum_loss:.3f}')

Epoch: 1
loss_TRAIN =1.413
Epoch: 2
loss_TRAIN =1.273
Epoch: 3
loss_TRAIN =1.254


## Multiple outputs for one sample

In [10]:
sample_input = "سخن هر چه زین گوهران بگذرد "#" کزین برتر اندیشه برنگذرد"
print(sample_input)
sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
sample_input_ids = sample_input_ids.to(device)

sample_outputs = model.generate(
    input_ids=sample_input_ids,
    do_sample=True,
    top_k=50,
    max_length=50,
    top_p=0.95,
    num_return_sequences=5
)
for i, sample_output in enumerate(sample_outputs):
    output = tokenizer.decode(sample_output, skip_special_tokens=False)
    output = output.replace("<|startoftext|>", "\n").replace("<s>", "").replace("</s>", "").replace("<sep>", "\n").replace("<pad>","")

    print(f'output: {output}')

سخن هر چه زین گوهران بگذرد 
output: سخن هر چه زین گوهران بگذرد 


 اندر شاه در بر
output: سخن هر چه زین گوهران بگذرد 
بهزبی او و را
output: سخن هر چه زین گوهران بگذرد 



ز که در
output: سخن هر چه زین گوهران بگذرد 


که از سخن به
output: سخن هر چه زین گوهران بگذرد ست

ز روان در تو باد


## Outputs for different inputs <br>
Note that the "Original beyt" Is actually the next "beyt", as we trained the model on this.

In [11]:
test_batch=next(iter(test_loader))
model.eval()
with torch.no_grad():
    (input_ids, attn_masks, output_ids)=test_batch
    input_ids=input_ids.to(device)
    for i in range(len(input_ids)):
        sample_input=input_ids[i]
        decoded_sample_input = tokenizer.decode(sample_input.cpu(), skip_special_tokens=False)
        cleaned_sample_input = decoded_sample_input.replace(tokenizer.pad_token, "").replace('</s>',"")
        print("Input beyt: \n",cleaned_sample_input)
        s=cleaned_sample_input.split('<sep>')[1]
        final_input=tokenizer(s)
        a1=torch.tensor(final_input.input_ids).to(device).unsqueeze(0)
        a2=torch.tensor(final_input.attention_mask).to(device).unsqueeze(0)
        print("Final: ", s + '   ')
        sample_output = model.generate(
                input_ids=a1,
                do_sample=True,
                top_k=50,
                max_length=80,
                top_p=0.95,
                num_return_sequences=1
            )
        decoded_sample_output = tokenizer.decode(sample_output.cpu()[0], skip_special_tokens=False)
        cleaned_sample_output = decoded_sample_output.replace(tokenizer.pad_token, "")
        print("Output beyt: \n",cleaned_sample_output)

        decoded_output = tokenizer.decode(output_ids[i].cpu(), skip_special_tokens=False)
        cleaned_output = decoded_output.replace(tokenizer.pad_token, "")
        print("Original output beyt: \n",cleaned_output)

Input beyt: 
 <s>گهی باده خورد و گهی تاخت اسپ<sep>بیامد سوی خان آذرگشسپ
Final:  بیامد سوی خان آذرگشسپ   
Output beyt: 
 بیامد سوی خان آذرگشسپ و برب<sep>ز اندران و رزم ب
Original output beyt: 
 <s>جهان آفرین را ستایش گرفت<sep>به آتشکده در نیایش گرفت</s>
Input beyt: 
 <s>ز شاهان برنای سیصد سوار<sep>همی راند با نامور شهریار
Final:  همی راند با نامور شهریار   
Output beyt: 
 همی راند با نامور شهریار و بود رزم<sep><sep> شاه و به را
Original output beyt: 
 <s>ابا یاره و طوق و زرین کمر<sep>بهر مهره ای در نشانده گهر</s>
Input beyt: 
 <s>که بهرام را پادشاهی و گنج<sep>ازان تو بیش است نابرده رنج
Final:  ازان تو بیش است نابرده رنج   
Output beyt: 
 ازان تو بیش است نابرده رنج<sep><sep><sep><sep>ز و را</s>
Original output beyt: 
 <s>پراز درد و غم شد ز تیمار اوی<sep>دلش گشت پیچان ز کردار اوی</s>
Input beyt: 
 <s>سوی شارستانها گشادست راه<sep>چه کهتر بدان مرز پوید چه شاه
Final:  چه کهتر بدان مرز پوید چه شاه   
Output beyt: 
 چه کهتر بدان مرز پوید چه شاه<sep> به پر را اندر یزدان</s>
Original output beyt

## Perplexity calculation

In [12]:
model.eval()
nlls= []
with torch.no_grad():
    for data in test_loader:
        (input_ids, attn_masks, output_ids)=data
        input_ids=input_ids.to(device)
        output_ids=output_ids.to(device)
        attn_masks=attn_masks.to(device)
        outputs = model(input_ids, labels=output_ids, attention_mask=attn_masks)
        nll_batch = outputs[0]
        nlls.append(nll_batch)

ppl = torch.exp(torch.stack(nlls).mean())
print('Model perplexity on test set: ',ppl)

Model perplexity on test set:  tensor(3.6151, device='cuda:0')
